In [1]:
import requests
import datetime
import re
from typing import Union

from tinyagents import chainable, respond, loop, passthrough

@chainable
class LegalResearcher:
    name: str = "legal_researcher"

    def __init__(self, llm, max_steps: int = 5):
        self.max_steps = max_steps
        self.prompt = """You are a highly capable AI legal researcher for the UK jurisdiction. You will be provided with a legal query from the user, and you break the query down into a set of legal problems or "questions" that needed to be investigated. Once you have concluded your investigation and analysis, you must respond with a final answer to the user. Your final answer must cite the documents (if any) that were used to guide your answer by referencing the document id in sqaure brackets such as ".. some statement [.. document ID goes here]".

        Below is an example of an interaction between you and the user:

        User Query: [.. user query will be provided here]

        Question: [.. provide your question here]
        Search results: [.. returned search results]
        Thought: [.. provide your thoughts on the above results here]

        You may repeat the process above up to {max_steps} times. 

        When you are finished with your analysis, you must provide your summary as such:

        Final answer: [.. provide your final answer here]
        
        Begin!

        User Query: {user_query}
        {history}"""
        self.llm = llm

    def run(self, state: Union[dict, str]):
        if not isinstance(state, dict):
            state = {
                "user_query": state,
                "steps": []
            }
        
        user_query = state["user_query"]
        history = self._get_history(state)
        output = self.llm.generate_content(self.prompt.format(history=history, max_steps=self.max_steps, user_query=user_query)).text
    
        if "Question:" in output:
            state["steps"].append(
                dict(question=output.split("Question: ")[-1]))
            
        if "Thought:" in output:
            last_step = state["steps"][-1]
            last_step["thought"] = output.split("Thought:")[-1]
            state["steps"][-1] = last_step
            
        if "Final answer:" in output:
            state["final_answer"] = self._replace_citations(output.split("Final answer: ")[-1], state)
        
        return state
    
    def output_handler(self, state: dict):
        if "final_answer" in state:
            return respond(state["final_answer"])
        
        return passthrough(state)
    
    @staticmethod
    def _replace_citations(output: str, state: dict):
        """ messy means of replacing document ID references with source link """
        citations_map = {}
        for step in state["steps"]:
            if not "results" in step:
                continue
            for id_, result in step["results"].items():
                citations_map[id_] = result["link"]
                
        citations = re.findall("\[(.*?)\]", output)
        for citation in citations:
            try:
                if "Document ID:" in citation:
                    citation = citation.replace("Document ID:", "").strip()
                    output = output.replace("Document ID: ", "")
                
                if "," in citation:
                    for citation_ in citation.split(","):
                        output = output.replace(output, citations_map[citation_.strip()])
                    continue

                if "www" in citation:
                    continue
                    
                output = output.replace(citation, citations_map[citation])
            except KeyError:
                output = output.replace(citation, "")

        return output
    
    def _get_history(self, state: dict):
        if len(state["steps"]) == 0:
            return "Question: "
        
        step_template ="""\n\nQuestion: {question}\nSearch Results: {results}\nThought: {thought}"""
        history = ""
        for step in state["steps"]:
            history += step_template.format(question=step["question"], results=self._format_results(step["results"]), thought=step.get("thought"))

        return history
    
    @staticmethod
    def _format_results(results: dict):
        return "\n\n".join([f"Document ID: {id_}\n\tTitle: {result['title']}\n\tSnippet: {result['snippet']}" for id_, result in results.items()])

@chainable
class SearchTool:
    name: str = "search_tool"

    def __init__(self, api_key: str, sites: list[str]):
        self.api_key = api_key
        self.sites = " OR ".join(["site:" + site for site in sites])

    def run(self, state: dict):
        for step in state["steps"]:
            if "results" not in step:
                step["results"] = self.get_results(step["question"])

        return state

    def get_results(self, query: str):
        payload =  {
            "url": "https://google.serper.dev/search",
            "headers": {
                "X-API-KEY": self.api_key,
            },
            "params": {
                "q": f"{self.sites} {query}"
            }
        }
        response = requests.get(**payload).json()

        results = {
            f"{datetime.datetime.now().strftime('%d%Y%H%M%S%fZ')}": {
                "snippet": result["snippet"],
                "link": result["link"],
                "title": result["title"]
            } for result in response["organic"]
        }
        return results

In [28]:
import os
import google.generativeai as genai
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

llm = genai.GenerativeModel(
    "gemini-1.5-flash", 
    generation_config=dict(
        stop_sequences=["Search results:"],
        max_tokens=1024
    )
)

agent = LegalResearcher(llm=llm, max_steps=3)

In [29]:
api_key = os.environ["SERPER_API_KEY"]

search_tool = SearchTool(api_key=api_key, sites=["handbook.fca.org.uk"])

In [30]:
graph = loop(agent, search_tool, name="agent", max_iter=4).as_graph()

runner = graph.compile()

In [31]:
query = "What is the definition of an e-money provider?"

output = runner.execute(query)


 > Running node: legal_researcher

	Input: What is the definition of an e-money provider?

	Output: {
  "content": {
    "user_query": "What is the definition of an e-money provider?",
    "steps": [
      {
        "question": "What are the legal requirements for an entity to be classified as an e-money provider in the UK?\n\n"
      }
    ]
  },
  "action": null,
  "ref": null
}

 > Running node: search_tool

	Input: {'user_query': 'What is the definition of an e-money provider?', 'steps': [{'question': 'What are the legal requirements for an entity to be classified as an e-money provider in the UK?\n\n'}]}

	Output: {
  "content": {
    "user_query": "What is the definition of an e-money provider?",
    "steps": [
      {
        "question": "What are the legal requirements for an entity to be classified as an e-money provider in the UK?\n\n",
        "results": {
          "252024183929091112Z": {
            "snippet": "(i) allow the holder to acquire goods or services only in th

In [33]:
from IPython.display import Markdown, display

display(Markdown(output))

In the UK, an e-money provider is defined by the Electronic Money Regulations 2011.  An entity is considered an e-money provider if it issues electronic money, which is defined as a digital form of money that is: 

1.  **Electronically stored** [https://www.handbook.fca.org.uk/instrument/2011/2011_7.pdf]
2.  **Accepted as a means of payment** [https://www.handbook.fca.org.uk/instrument/2011/2011_7.pdf]
3.  **Issued on receipt of funds** [https://www.handbook.fca.org.uk/instrument/2011/2011_7.pdf]
4.  **Does not represent a claim on the issuer** [https://www.handbook.fca.org.uk/instrument/2011/2011_7.pdf]

E-money providers must be authorized or registered by the Financial Conduct Authority (FCA) to operate in the UK. [https://www.handbook.fca.org.uk/handbook/PERG/15/?view=chapter] The FCA sets out specific requirements for e-money providers, including capital adequacy and financial crime prevention measures. [https://www.handbook.fca.org.uk/handbook/PERG/15/?view=chapter]  
